In [1]:
#いろいろインポート
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ
place='kiryu'

In [2]:
#モデルスコアデータの読み込み
score_df = pd.read_csv('../csv/model_score_date_{}.csv'.format(place))
score_df=score_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
display(score_df)

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,374060.0,423200.0,308.0,4232.0,88.388469,7.277883,308.0
1,1.0,4.0,52.0,374060.0,423100.0,308.0,4231.0,88.409359,7.279603,308.0
2,1.0,4.0,53.0,374060.0,423100.0,308.0,4231.0,88.409359,7.279603,308.0
3,1.0,4.0,54.0,374060.0,422800.0,308.0,4228.0,88.472091,7.284768,308.0
4,1.0,4.0,55.0,374060.0,422800.0,308.0,4228.0,88.472091,7.284768,308.0
...,...,...,...,...,...,...,...,...,...,...
13855,28.0,8.0,145.0,6650.0,18300.0,20.0,183.0,36.338798,1.639344,3.0
13856,28.0,8.0,146.0,6650.0,17700.0,20.0,177.0,37.570621,1.694915,3.0
13857,28.0,8.0,147.0,6650.0,17700.0,20.0,177.0,37.570621,1.694915,3.0
13858,28.0,8.0,148.0,6650.0,15000.0,20.0,150.0,44.333333,2.000000,3.0


#### モデル性能評価用のデータフレーム作成(result_comごとの閾値を持つ)


In [3]:
#学習データの読み込み
result_df = pd.read_csv('../csv/train_{}_date.csv'.format(place))
result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除

df=result_df
#学習、テストデータ切り分け
num_data = len(df) 
num_valid = int(num_data/6.0)
valid_df = df.loc[(num_data-num_valid):num_data, :]#なるべく最新のデータを未知データに
train_df = df.loc[0:(num_data-num_valid-1), :]
train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])
#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
display(result_com_df)

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1,228,7.183365,1220.877193,910.0
1,2,157,4.946440,1595.796178,1090.0
2,3,122,3.843730,1621.885246,1245.0
3,4,71,2.236925,3579.295775,2020.0
4,5,145,4.568368,1479.172414,1170.0
...,...,...,...,...,...
113,116,2,0.063012,13755.000000,13755.0
114,117,4,0.126024,13812.500000,11345.0
115,118,4,0.126024,23727.500000,23980.0
116,119,4,0.126024,19122.500000,15525.0


### モデル性能評価機能

In [4]:
#基準をクリアしたモデルを格納するdf
depths_arr=[4,5,6,7,8]
model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for target_com in result_com_df['result_com'].values:
    target_com_df=score_df[score_df['target_com']==target_com]
    #閾値の作成===============================================================
    gain_th=110#利益率の閾値
    result_s=result_com_df[result_com_df['result_com']==target_com]
    buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
    num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
    
    diff_gain_th=108#利益率の閾値
    diff_buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#前後モデルの買ったうちの的中率の閾値
    diff_num_tp_th=result_s['result_com_num'].values[0]*0.13#前後モデルのあたった回数の閾値(出現回数の20%が的中)
    
    #==========================================================
    for target_depth in depths_arr:
        target_com_depth_df=target_com_df[target_com_df['depth']==target_depth]
        
        len_df=len(target_com_depth_df)
        df=target_com_depth_df.copy()
        df['number_i']=np.arange(0,len_df,1)
        
        for _, row in df.iterrows():
            if ((row['number_i']==0) or (row['number_i']==1) or (row['number_i']==len_df-2) or (row['number_i']==len_df-1)):
                pass
            else:
                #前後のモデルのスコア===================================
                diff_m2_row=df[df['number_i']==(row['number_i']-2)]
                diff_m1_row=df[df['number_i']==(row['number_i']-1)]
                diff_p1_row=df[df['number_i']==(row['number_i']+1)]
                diff_p2_row=df[df['number_i']==(row['number_i']+2)]
                diff_models=[diff_m2_row,diff_m1_row,diff_p1_row,diff_p2_row]
                #===================================================
                if ((row['利益率']>gain_th) and (row['購買的中率']>buy_accuracy_th) and (row['的中数']>=num_tp_th)):#初めに真ん中のモデルの性能評価
                    #前後モデルの評価
                    flag_arr=[0,0,0,0]
                    for index in range(len(flag_arr)):

                        diff_model=diff_models[index].iloc[0,:]
                        if ((diff_model['利益率']>diff_gain_th) and (diff_model['購買的中率']>diff_buy_accuracy_th) and (diff_model['的中数']>=diff_num_tp_th)):#前後モデルの性能評価
                            flag_arr[index]=1
                        else:
                            pass
                    if sum(flag_arr)==4:
                        model_para_df=model_para_df.append(row)
                        
                else:
                    pass
        

In [5]:
model_para_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
1062,3.0,4.0,123.0,25260.0,18700.0,162.0,187.0,135.080214,16.577540,31.0,72.0
1165,3.0,5.0,127.0,24310.0,21600.0,162.0,216.0,112.546296,14.351852,31.0,76.0
1375,3.0,7.0,139.0,21230.0,17800.0,162.0,178.0,119.269663,15.168539,27.0,88.0
1376,3.0,7.0,140.0,24470.0,20600.0,162.0,206.0,118.786408,15.533981,32.0,89.0
1477,3.0,8.0,142.0,26730.0,20900.0,162.0,209.0,127.894737,15.789474,33.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...
10769,22.0,7.0,128.0,48170.0,42900.0,63.0,429.0,112.284382,6.060606,26.0,77.0
10770,22.0,7.0,129.0,44840.0,36300.0,63.0,363.0,123.526171,6.611570,24.0,78.0
10871,22.0,8.0,131.0,47020.0,39800.0,63.0,398.0,118.140704,6.281407,25.0,80.0
10872,22.0,8.0,132.0,46450.0,37900.0,63.0,379.0,122.559367,6.860158,26.0,81.0


In [6]:
model_para_df.to_csv('../csv/{}_good_score_model_date.csv'.format(place), encoding='utf_8_sig')

In [7]:
diff_model

target_com      27.0
depth            5.0
target_per     149.0
総収益           1220.0
投資金額          2000.0
出現数             20.0
購買予測数           20.0
利益率             61.0
購買的中率            5.0
的中数              1.0
number_i        98.0
Name: 13067, dtype: float64

In [17]:
len_df

99

In [34]:
type(diff_model)

tuple